In [1]:
import os
import tempfile
import shutil
import pathlib

from freezegun import freeze_time

# import to register magics
import sqlmesh

In [2]:
freezer = freeze_time("2032-01-01 00:00:00")
freezer.start()

In [3]:
root_dir = pathlib.Path(os.getcwd()).parents[2]
EXAMPLE_SUSHI_DIR = pathlib.Path(root_dir) / "examples" / "sushi"
str(EXAMPLE_SUSHI_DIR)

'/home/butch2/play/study/learn-sqlmesh/sqlmesh2/examples/sushi'

In [4]:
TARGET_SUSHI_DIR = pathlib.Path(tempfile.gettempdir()) / "test_local_jupyter_notebook"
str(TARGET_SUSHI_DIR)

'/tmp/test_local_jupyter_notebook'

In [5]:
TARGET_MODEL_DIR = TARGET_SUSHI_DIR / "models"
str(TARGET_MODEL_DIR)

'/tmp/test_local_jupyter_notebook/models'

In [6]:
def ignore(src, names):
    if pathlib.Path(src).name in {".cache", "__pycache__", "logs", "data"}:
        return names
    return []

if TARGET_SUSHI_DIR.exists():
    shutil.rmtree(str(TARGET_SUSHI_DIR))
shutil.copytree(str(EXAMPLE_SUSHI_DIR), str(TARGET_SUSHI_DIR), ignore=ignore)

'/tmp/test_local_jupyter_notebook'

In [7]:
%context --config local_config $TARGET_SUSHI_DIR 

SQLMesh project context set to: /tmp/test_local_jupyter_notebook

### Leaving plan unapplied to verify that the text boxes are printed

In [8]:
%cd $TARGET_SUSHI_DIR

/tmp/test_local_jupyter_notebook


/home/butch2/play/study/learn-sqlmesh/sqlmesh2/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
# %plan

In [10]:
%plan --no-prompts --auto-apply

/home/butch2/play/study/learn-sqlmesh/sqlmesh2/sqlmesh/core/test/definition.py:185: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expected[col] = pd.to_datetime(expected[col], errors="ignore").dt.date  # type: ignore


HTML(value='<span style="font-size: 11px; font-weight: bold; font-family: Menlo,\'DejaVu Sans Mono\',consolas,…

New environment `prod` will be created from `prod`

Summary of differences against `prod`:

Models:
└── Added:
    ├── sushi.marketing
    ├── sushi.customers
    ├── sushi.customer_revenue_by_day
    ├── sushi.top_waiters
    ├── sushi.order_items
    ├── sushi.waiters
    ├── sushi.waiter_names
    ├── sushi.waiter_as_customer_by_day
    ├── raw.model1
    ├── sushi.orders
    ├── sushi.items
    ├── sushi.customer_revenue_lifetime
    ├── sushi.raw_marketing
    ├── sushi.waiter_revenue_by_day
    ├── raw.model2
    └── raw.demographics

...

All model versions have been created successfully

Output()

All model batches have been executed successfully

Output()

The target environment has been updated successfully

In [11]:
%plan dev --no-prompts --auto-apply --include-unmodified

/home/butch2/play/study/learn-sqlmesh/sqlmesh2/sqlmesh/core/test/definition.py:185: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expected[col] = pd.to_datetime(expected[col], errors="ignore").dt.date  # type: ignore


HTML(value='<span style="font-size: 11px; font-weight: bold; font-family: Menlo,\'DejaVu Sans Mono\',consolas,…

New environment `dev` will be created from `prod`

Output()

All model versions have been created successfully

Output()

The target environment has been updated successfully

Virtual Update executed successfully

In [12]:
%render sushi.top_waiters

SELECT                                                                                                             
  CAST("waiter_revenue_by_day"."waiter_id" AS INT) AS "waiter_id",                                                 
  CAST("waiter_revenue_by_day"."revenue" AS DOUBLE) AS "revenue"                                                   
FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*              
local.sushi.waiter_revenue_by_day */                                                                               
WHERE                                                                                                              
  "waiter_revenue_by_day"."event_date" = (                                                                         
    SELECT                                                                                                         
      MAX("waiter_revenue_by_day"."event_date") AS "_col_0"                                                        
    FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*          
local.sushi.waiter_revenue_by_day */                                                                               
  )                                                                                                                
ORDER BY                                                                                                           
  "revenue" DESC                                                                                                   
LIMIT 10                                                                                                           

In [13]:
%render sushi.top_waiters --no-format

SELECT
  CAST("waiter_revenue_by_day"."waiter_id" AS INT) AS "waiter_id",
  CAST("waiter_revenue_by_day"."revenue" AS DOUBLE) AS "revenue"
FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /* 
local.sushi.waiter_revenue_by_day */
WHERE
  "waiter_revenue_by_day"."event_date" = (
    SELECT
      MAX("waiter_revenue_by_day"."event_date") AS "_col_0"
    FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /* 
local.sushi.waiter_revenue_by_day */
  )
ORDER BY
  "revenue" DESC
LIMIT 10

In [14]:
%evaluate sushi.top_waiters

,waiter_id,revenue
0,4,317.39
1,0,273.09
2,2,250.84
3,7,155.52
4,5,124.39
5,1,60.18
6,9,48.86
7,6,21.52


In [ ]:
# top_waiters = TARGET_MODEL_DIR / "top_waiters.sql"
# top_waiters.write_text("""/* View of top waiters. */
# MODEL (
#   name sushi.top_waiters,
#   owner jen,
#   audits (
#     unique_values(columns=[waiter_id])
#   ),
#   grain waiter_id
# );

# SELECT
#   waiter_id::INT AS waiter_id,
#   revenue::DOUBLE AS revenue,
#   1 as new_col
# FROM sushi.waiter_revenue_by_day
# WHERE
#   event_date = (
#     SELECT
#       MAX(event_date)
#     FROM sushi.waiter_revenue_by_day
#   )
# ORDER BY
#   revenue DESC
# LIMIT 10
# """)

In [18]:
%%model sushi.top_waiters
/* View of top waiters. */
MODEL (
  name sushi.top_waiters,
  owner jen,
  dialect duckdb,
  audits (UNIQUE_VALUES('columns' = (
    waiter_id
  ))),
  grains (waiter_id)
);

SELECT
  waiter_id::INT AS waiter_id,
  revenue::DOUBLE AS revenue,
  1::INT AS new_col
FROM sushi.waiter_revenue_by_day
WHERE
  event_date = (
    SELECT
      MAX(event_date)
    FROM sushi.waiter_revenue_by_day
  )
ORDER BY
  revenue DESC
LIMIT 10

Model `sushi.top_waiters` updated

SELECT                                                                                                             
  CAST("waiter_revenue_by_day"."waiter_id" AS INT) AS "waiter_id",                                                 
  CAST("waiter_revenue_by_day"."revenue" AS DOUBLE) AS "revenue",                                                  
  CAST(1 AS INT) AS "new_col"                                                                                      
FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*              
local.sushi.waiter_revenue_by_day */                                                                               
WHERE                                                                                                              
  "waiter_revenue_by_day"."event_date" = (                                                                         
    SELECT                                                                                                         
      MAX("waiter_revenue_by_day"."event_date") AS "_col_0"                                                        
    FROM "local"."sqlmesh__sushi"."sushi__waiter_revenue_by_day__200393305" AS "waiter_revenue_by_day" /*          
local.sushi.waiter_revenue_by_day */                                                                               
  )                                                                                                                
ORDER BY                                                                                                           
  "revenue" DESC                                                                                                   
LIMIT 10                                                                                                           

In [19]:
%diff prod

Summary of differences against `prod`:

Models:
└── Directly Modified:
    └── sushi.top_waiters

In [23]:
%plan dev --no-prompts --auto-apply

/home/butch2/play/study/learn-sqlmesh/sqlmesh2/sqlmesh/core/test/definition.py:185: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expected[col] = pd.to_datetime(expected[col], errors="ignore").dt.date  # type: ignore


HTML(value='<span style="font-size: 11px; font-weight: bold; font-family: Menlo,\'DejaVu Sans Mono\',consolas,…

Summary of differences against `dev`:

Models:
└── Directly Modified:
    └── sushi__dev.top_waiters

---                                                                                                                
                                                                                                                   
+++                                                                                                                
                                                                                                                   
@@ -9,7 +9,8 @@                                                                                                    
                                                                                                                   
 )                                                                                                                 
 SELECT                                                                                                            
   CAST(waiter_id AS INT) AS waiter_id,                                                                            
-  CAST(revenue AS DOUBLE) AS revenue                                                                              
+  CAST(revenue AS DOUBLE) AS revenue,                                                                             
+  CAST(1 AS INT) AS new_col                                                                                       
 FROM sushi.waiter_revenue_by_day                                                                                  
 WHERE                                                                                                             
   event_date = (                                                                                                  

...

Output()

All model versions have been created successfully

All model batches have been executed successfully

Output()

The target environment has been updated successfully

In [24]:
!mkdir -p $TARGET_SUSHI_DIR/html

In [25]:
%dag --file $TARGET_SUSHI_DIR/html/dag.html

In [30]:
from IPython.display import IFrame

In [42]:
IFrame(src='./dag.html', width=900, height=800)

In [43]:
%migrate

Migration complete

In [44]:
%table_diff prod:dev --model sushi.top_waiters

Schema Diff Between 'PROD' and 'DEV' environments for model 'sushi.top_waiters':
└── Added Columns:
    └── new_col (INT)

Row Counts:
├──  COMMON: 8 rows
├──  PROD ONLY: 0 rows
└──  DEV ONLY: 0 rows

COMMON ROWS column comparison stats:

pct_match
revenue      100.0

In [45]:
%run_test {TARGET_SUSHI_DIR}/tests/test_customer_revenue_by_day.yaml::test_customer_revenue_by_day


----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


In [46]:
%audit sushi.top_waiters

Found 1 audit(s).

unique_values on model sushi.top_waiters ✅ PASS.

Finished with 0 audit errors and 0 audits skipped.

Done.

In [47]:
%%fetchdf my_result
SELECT 1

,1
0,1


In [48]:
my_result

,1
0,1


In [49]:
%info

Models: 16

Macros: 3

Data warehouse connection succeeded

Test connection succeeded

In [50]:
%migrate

Migration complete

In [51]:
freezer = freeze_time("2032-01-02 00:00:00")
freezer.start()

In [52]:
%run_dag

Output()

All model batches have been executed successfully

Run finished for environment 'prod'

In [ ]:
# %invalidate dev

In [ ]:
# %context

In [ ]:
import sqlmesh

In [ ]:
%create_test?

In [ ]:
%lsmagic?

In [ ]:
%lsmagic

In [ ]:
%%model sushi.top_waiters
/* View of top waiters. */
MODEL (
  name sushi.top_waiters,
  owner jen,
  dialect duckdb,
  audits (UNIQUE_VALUES('columns' = [waiter_id])),
  grains (waiter_id)
);

SELECT
  waiter_id::INT AS waiter_id,
  revenue::DOUBLE AS revenue,
  1 AS new_col,
  2 AS new_col2
FROM sushi.waiter_revenue_by_day
WHERE
  event_date = (
    SELECT
      MAX(event_date)
    FROM sushi.waiter_revenue_by_day
  )
ORDER BY
  revenue DESC
LIMIT 100

In [ ]:
%%model sushi.top_waiters
/* View of top waiters. */
MODEL (
  name sushi.top_waiters,
  owner jen,
  dialect duckdb,
  audits (UNIQUE_VALUES('columns' = [waiter_id])),
  grains (waiter_id)
);

SELECT
  waiter_id::INT AS waiter_id,
  revenue::DOUBLE AS revenue,
  1 AS new_col,
  2 AS new_col2,
  3 AS new_col3
FROM sushi.waiter_revenue_by_day
WHERE
  event_date = (
    SELECT
      MAX(event_date)
    FROM sushi.waiter_revenue_by_day
  )
ORDER BY
  revenue DESC
LIMIT 100